<a href="https://colab.research.google.com/github/rrrepsac/nmt/blob/master/translation_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1y3Qzjd89Evpiai2MCbkOuE5aMu-87aI2'
!unzip pl-en.zip

Downloading...
From: https://drive.google.com/uc?id=1y3Qzjd89Evpiai2MCbkOuE5aMu-87aI2
To: /content/pl-en.zip
61.8MB [00:02, 22.7MB/s]
Archive:  pl-en.zip
   creating: pl-en/
  inflating: pl-en/europarl-v7.pl-en.en  
  inflating: pl-en/europarl-v7.pl-en.pl  


In [26]:
ds_name=''#'pl-en'

In [27]:
def prepare_corpus(root_path, max_lines=-1):
    files = []
    timer = Timer()
    for root, dirs, cur_files in os.walk(root_path):
        if 'europar' in cur_files[0]:
            files = [Path(root)/file for file in cur_files]
            break
    # print(files)
    lines_number = 0
    cn = 0
    all = 0
    sentences_en = []
    sentences_pl = []
    with open(files[0], 'r', encoding='utf-8') as f_en:
        with open(files[1], 'r', encoding='utf-8') as f_pl:
            while True:
                line_en = f_en.readline()
                line_pl = f_pl.readline()
                if line_en == '' or line_en == '':
                    timer.get(f'{lines_number}, {cn} {line_en} {line_pl}')
                    break
                line_en = line_en.strip()
                line_pl = line_pl.strip()
                if line_en.endswith('('):
                    line_en = line_en[:-1]
                if line_pl.endswith('('):
                    line_pl = line_pl[:-1]
                all += 1
                if line_en == line_pl:
                    continue
                if len(line_en) < 10 or len(line_pl) < 10:
                    continue
                p = len(line_pl) / len(line_en)
                if p < 1:
                    p = 1/p
                if p > 2:
                    continue
                    # print(all, lines_number, line_en.strip(), f'len={len(line_en.strip())}', '__', line_pl.strip(), f'len = {len(line_pl.strip())}', '<<')
                    # cn += 1
                sentences_en.append(line_en)
                sentences_pl.append(line_pl)
                lines_number += 1
                if max_lines > 0 and lines_number > max_lines:
                    break
    with open('norm.en', 'w') as fn_en:
        print(*sentences_en, sep='\n', file=fn_en)
    with open('norm.pl', 'w') as fn_pl:
        print(*sentences_pl, sep='\n', file=fn_pl)
    return
def sort_parallel_corpus(from_list, to_list):
    sorted_positions = sorted(range(len(from_list)), key=lambda i: len(from_list[i]) + len(to_list[i]))
    coinc_pos = 0
    for i, (c,n) in enumerate(zip(sorted_positions[:-1], sorted_positions[1:])):
        if from_list[c] == from_list[n] and to_list[c] == to_list[n]:
            sorted_positions.pop(i - coinc_pos)
            coinc_pos += 1
    print('coinc: ', coinc_pos)

    return [from_list[i] for i in sorted_positions], [to_list  [i] for i in sorted_positions]




In [28]:
import os
from pathlib import Path
import time
from torch.utils.data import random_split
from functools import wraps

# def timing(f):
#     @wraps(f)
#     def wrapper(*args, **kwargs):
#         t_beg = time.time()
#         result = f(*args, **kwargs)
#         t_end = time.time()
#         print(f'{f.__name__} t = {t_end - t_beg}')
#         return result
#     return wrapper

# class Timer:
#     def __init__(self, message=''):
#         self.start_time = time.time()
#         self.last_time = self.start_time
    
#     def get(self, message='', update_time=True):
#         t = time.time()
#         print(f'{message}, dt = {t - self.last_time:4.3f}, all_time = {t - self.start_time: 5.3}', flush=True)
#         if update_time:
#             self.last_time = t


In [29]:
def my_iter(split_name='train', src='norm.pl', tgt='norm.en'):
  src_seq = []
  tgt_seq = []
  with open(src, 'r') as f:
    src_seq = f.readlines()
  with open(tgt, 'r') as f:
    tgt_seq = f.readlines()
  l = len(src_seq)
  ratios = [0.85, 0.1, 0.5]
  borders= [0, int(ratios[0]*l), int(l*sum(ratios[:2])), l]
  slices = {
      'train': slice(borders[0], borders[1]),
      'test' : slice(borders[1], borders[2]),
      'valid': slice(borders[2], borders[3])
  }
  print('doing', split_name, slices[split_name])
  for x, y in zip(src_seq[slices[split_name]], tgt_seq[slices[split_name]]):
    yield (x, y)

In [30]:
prepare_corpus('pl-en', 100000)


In [31]:
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# !python -m spacy download de_core_news_sm
!pip install youtokentome

In [32]:
from torchtext.datasets import Multi30k

def get_iter(split_name, SRC_LANGUAGE='de', TGT_LANGUAGE='en', from_ds=None):
  if from_ds != 'pl-en':
    return Multi30k(split=split_name, language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  else:
    return my_iter(split_name)

In [33]:
import youtokentome as yttm
train_iter = get_iter('train', from_ds='pl-en')
for x in train_iter:
  print(x)
  break
sseqs = []
tseqs = []
for src, tgt in train_iter:
  sseqs.append(src)
  tseqs.append(tgt)
with open('src.txt', 'w') as f:
  print(*sseqs, sep='', end='', file=f)
with open('tgt.txt', 'w') as f:
  print(*tseqs, sep='', end='', file=f)



doing train slice(0, 85000, None)
("Action taken on Parliament's resolutions: see Minutes\n", 'Działania podjęte w wyniku rezolucji Parlamentu: Patrz protokól\n')


In [34]:
voc_size = 10000
src_bpe = yttm.BPE.train('src.txt', 'src.model', voc_size, 0.99)
tgt_bpe = yttm.BPE.train('tgt.txt', 'tgt.model', voc_size, 0.99)


In [35]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

# Define special symbols and indices
PAD_IDX, UNK_IDX,  BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<pad>', '<unk>', '<bos>', '<eos>']
 

train_iter = get_iter('train')

def my_collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = [torch.tensor(x) for x in src_bpe.encode(src_batch, bos=True, eos=True)]
    tgt_batch = [torch.tensor(x) for x in tgt_bpe.encode(tgt_batch, bos=True, eos=True)]

    # print(src_batch,'\n', tgt_batch)
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch
for x, y in DataLoader(train_iter, batch_size=2, collate_fn=my_collate_fn):
  # print(x, y)
  # print(tgt_bpe.decode(y.T[0].tolist()))
  break


In [36]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network 
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [37]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [38]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = src_bpe.vocab_size()#len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = tgt_bpe.vocab_size()#len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = min(128, 512)
NHEAD = 16
FFN_HID_DIM = min(EMB_SIZE*4, 512)
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [39]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = get_iter('train')
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=my_collate_fn)
    
    for i, (src, tgt) in enumerate(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        # print(src.shape, tgt.shape)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        if i % 20 == 0:
          print('loss=', loss.item())
        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = get_iter('valid')
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=my_collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

Now we have all the ingredients to train our model. Let's do it!




In [40]:
for p in transformer.named_parameters():
  break
  # print(p[0], p[1].shape)


In [52]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))



loss= 2.9094691276550293
loss= 2.984081268310547
loss= 3.0194807052612305
loss= 2.877934455871582
loss= 2.9021739959716797
loss= 2.857475757598877
loss= 2.940941333770752
loss= 2.812286615371704
loss= 2.7882628440856934
loss= 2.7479100227355957
loss= 2.9721012115478516
loss= 3.0513062477111816
Epoch: 1, Train loss: 2.877, Val loss: 2.702, Epoch time = 38.944s
loss= 2.697971820831299
loss= 2.791795492172241
loss= 2.791020393371582
loss= 2.7003588676452637
loss= 2.731104612350464
loss= 2.688497304916382
loss= 2.765437364578247
loss= 2.641385078430176
loss= 2.624960422515869
loss= 2.5938894748687744
loss= 2.8128561973571777
loss= 2.8935546875
Epoch: 2, Train loss: 2.701, Val loss: 2.540, Epoch time = 38.920s
loss= 2.559875249862671
loss= 2.663275957107544
loss= 2.6226212978363037
loss= 2.554858684539795
loss= 2.5928895473480225
loss= 2.5263795852661133
loss= 2.6491987705230713
loss= 2.51501727104187
loss= 2.5163612365722656
loss= 2.4621164798736572
loss= 2.696453094482422
loss= 2.76620268

KeyboardInterrupt: ignored

In [53]:
# actual function to translate input sentence into target language

# function to generate output sequence using greedy algorithm 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def my_translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    # src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)

    src = torch.tensor(src_bpe.encode([src_sentence], bos=True, eos=True)[0]).view(-1,1)
    # print(src.shape, ' = src_shape')
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    # print(tgt_tokens.shape)
    return tgt_bpe.decode(tgt_tokens.view(-1,1).tolist())
    #return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")


In [58]:
# print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))
for i, (src, tgt) in enumerate(get_iter('test', from_ds=ds_name)):
  print(src.strip())
  print(" ".join(my_translate(transformer, src)))
  print(tgt)
  if i > 3:
    break

Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.
<BOS>  <UNK> ma n in a b lu e s hi r t i s si t ing on a b lu e . <EOS>
A man in an orange hat starring at something.

Ein Boston Terrier läuft über saftig-grünes Gras vor einem weißen Zaun.
<BOS>  <UNK> y o un g g i r l in a b la ck s hi r t i s s h o ki ng a s a s a c row d in t he ba ck . <EOS>
A Boston Terrier is running on lush green grass in front of a white fence.

Ein Mädchen in einem Karateanzug bricht einen Stock mit einem Tritt.
<BOS>  <UNK> g i r l in a b la ck s hi r t i s in a b lu e s hi r t i s s hi r t ing . <EOS>
A girl in karate uniform breaking a stick with a front kick.

Fünf Leute in Winterjacken und mit Helmen stehen im Schnee mit Schneemobilen im Hintergrund.
<BOS> T he re i s pla y ing in a b la ck and s hi r t and s h o t and s h o t and b la ck s in t he i r t and s om e th ing . <EOS>
Five people wearing winter jackets and helmets stand in the snow, with snowmobiles in the background.

Leute Reparier

References
----------

1. Attention is all you need paper.
   https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
2. The annotated transformer. https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding

